In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler

Домашнее задание 4:
Таблица сопряженности

## 1. Формирование трех номинальных признаков х1, х2 и х3

In [ ]:
# Повторная загрузка библиотеки pandas и данных после сброса состояния выполнения
import pandas as pd

# Загрузка данных
df = pd.read_csv('country_wise_latest.csv')

# Признак х1 - 'WHO Region'
x1 = df['WHO Region']

# Определение пороговых значений для 'Confirmed' и 'Deaths'
confirmed_thresholds = [df['Confirmed'].quantile(q) for q in [0.33, 0.66]]
deaths_thresholds = [df['Deaths'].quantile(q) for q in [0.33, 0.66]]

# Категоризация 'Confirmed' (х2) и 'Deaths' (х3) на основе пороговых значений
x2 = pd.cut(df['Confirmed'], bins=[-1] + confirmed_thresholds + [df['Confirmed'].max()], labels=['Low', 'Medium', 'High'])
x3 = pd.cut(df['Deaths'], bins=[-1] + deaths_thresholds + [df['Deaths'].max()], labels=['Low', 'Medium', 'High'])

# Создание DataFrame для новых признаков
nominal_features = pd.DataFrame({'WHO_Region': x1, 'Confirmed_Cat': x2, 'Deaths_Cat': x3})
nominal_features.head()


,WHO_Region,Confirmed_Cat,Deaths_Cat
0,Eastern Mediterranean,High,High
1,Europe,Medium,Medium
2,Africa,High,High
3,Europe,Low,Medium
4,Africa,Low,Medium


In [ ]:
df.head()

,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,Confirmed last week,1 week change,1 week % increase,WHO Region
0,Afghanistan,36263,1269,25198,9796,106,10,18,3.50,69.49,5.04,35526,737,2.07,Eastern Mediterranean
1,Albania,4880,144,2745,1991,117,6,63,2.95,56.25,5.25,4171,709,17.00,Europe
2,Algeria,27973,1163,18837,7973,616,8,749,4.16,67.34,6.17,23691,4282,18.07,Africa
3,Andorra,907,52,803,52,10,0,0,5.73,88.53,6.48,884,23,2.60,Europe
4,Angola,950,41,242,667,18,1,0,4.32,25.47,16.94,749,201,26.84,Africa


х1: 'WHO Region' — категориальный признак из датасета.

х2: 'Confirmed' — категоризирован на 'Low', 'Medium', 'High'.

х3: 'Deaths' — также категоризирован на 'Low', 'Medium', 'High'.

## 2. Формирование таблиц сопряженности и расчет условных вероятностей

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

# Шаг 2: Формирование таблиц сопряженности и расчет условных вероятностей

# Таблица сопряженности для х1 и х2
contingency_table_x1_x2 = pd.crosstab(nominal_features['WHO_Region'], nominal_features['Confirmed_Cat'])

# Таблица сопряженности для х1 и х3
contingency_table_x1_x3 = pd.crosstab(nominal_features['WHO_Region'], nominal_features['Deaths_Cat'])

# Расчет условных вероятностей для х1 от х2
conditional_prob_x1_given_x2 = contingency_table_x1_x2.div(contingency_table_x1_x2.sum(axis=1), axis=0)

# Расчет условных вероятностей для х1 от х3
conditional_prob_x1_given_x3 = contingency_table_x1_x3.div(contingency_table_x1_x3.sum(axis=1), axis=0)

# Расчет коэффициентов Кетле
cramers_v_x1_x2 = np.sqrt(chi2_contingency(contingency_table_x1_x2)[0] / contingency_table_x1_x2.values.sum())
cramers_v_x1_x3 = np.sqrt(chi2_contingency(contingency_table_x1_x3)[0] / contingency_table_x1_x3.values.sum())

!!!

Описать как получились условные вероятности

Таблица сопряженности для х1 и х2

In [ ]:
contingency_table_x1_x2

Confirmed_Cat,Low,Medium,High
WHO_Region,,,
Africa,22,22,4
Americas,14,7,14
Eastern Mediterranean,4,6,12
Europe,10,20,26
South-East Asia,3,3,4
Western Pacific,9,3,4


Таблица сопряженности для х1 и х3

In [ ]:
contingency_table_x1_x3

Deaths_Cat,Low,Medium,High
WHO_Region,,,
Africa,21,23,4
Americas,14,6,15
Eastern Mediterranean,2,10,10
Europe,10,17,29
South-East Asia,5,2,3
Western Pacific,10,3,3


Расчет условных вероятностей для х1 от х2

Матрица условных вероятностей

Что Такое Условные Вероятности?
Условные вероятности позволяют оценить вероятность наступления одного события при условии, что произошло другое событие. В контексте таблиц сопряженности это означает оценку вероятности принадлежности к определенной категории одной переменной, учитывая категорию другой переменной.

Как Рассчитываются Условные Вероятности?
Условные вероятности рассчитываются путем деления количества событий, попадающих в определенную категорию одной переменной, на общее количество событий в категории другой переменной. Это позволяет оценить вероятность наличия одного признака при условии, что известен другой признак.

In [ ]:
conditional_prob_x1_given_x2

Confirmed_Cat,Low,Medium,High
WHO_Region,,,
Africa,0.458333,0.458333,0.083333
Americas,0.400000,0.200000,0.400000
Eastern Mediterranean,0.181818,0.272727,0.545455
Europe,0.178571,0.357143,0.464286
South-East Asia,0.300000,0.300000,0.400000
Western Pacific,0.562500,0.187500,0.250000


Расчет условных вероятностей для х1 от х3

Матрица условных вероятностей

In [ ]:
conditional_prob_x1_given_x3

Deaths_Cat,Low,Medium,High
WHO_Region,,,
Africa,0.437500,0.479167,0.083333
Americas,0.400000,0.171429,0.428571
Eastern Mediterranean,0.090909,0.454545,0.454545
Europe,0.178571,0.303571,0.517857
South-East Asia,0.500000,0.200000,0.300000
Western Pacific,0.625000,0.187500,0.187500


Для x1 и x2 (WHO Region и Confirmed Category):

Вы делите каждое значение в строке таблицы contingency_table_x1_x2 на сумму значений в этой строке. Это дает вам вероятность того, что страна в данном регионе ВОЗ попадает в каждую из категорий подтвержденных случаев.
Для x1 и x3 (WHO Region и Deaths Category):

Аналогично, вы делите каждое значение в строке таблицы contingency_table_x1_x3 на сумму значений в этой строке. Это показывает вероятность того, что страна в данном регионе ВОЗ попадает в каждую из категорий смертей.

Расчет коэффициентов Кетле

Коэффициент Кетле — это мера ассоциации между двумя категориальными переменными, и он обычно представляется в виде одного числа, а не в виде таблицы.

In [ ]:
cramers_v_x1_x2

0.41234567148434564

In [ ]:
cramers_v_x1_x3

0.46688542159556895

Таблица сопряженности для 'WHO Region' (х1) и категорий 'Confirmed' (х2).

Таблица сопряженности для 'WHO Region' (х1) и категорий 'Deaths' (х3).

Таблица сопряженности для х1 и х2: Показывает распределение категорий подтвержденных случаев ('Confirmed') по регионам ВОЗ.

Таблица сопряженности для х1 и х3: Демонстрирует распределение категорий смертей ('Deaths') по регионам ВОЗ.

Условные вероятности для х1 от х2 и х1 от х3: Предоставляют информацию о вероятности нахождения в определенном регионе при известной категории 'Confirmed' или 'Deaths'.

но если вам нужна таблица -

In [ ]:
# Создание DataFrame для хранения коэффициентов Кетле
cramers_v_table = pd.DataFrame(columns=x2.cat.categories, index=x3.cat.categories)

# Расчет коэффициентов Кетле для каждой пары категорий
for col in cramers_v_table.columns:
    for idx in cramers_v_table.index:
        contingency = pd.crosstab(nominal_features[nominal_features['Confirmed_Cat'] == col]['WHO_Region'],
                                  nominal_features[nominal_features['Deaths_Cat'] == idx]['WHO_Region'])
        cramers_v = np.sqrt(chi2_contingency(contingency)[0] / sum(contingency.sum()))
        cramers_v_table.at[idx, col] = cramers_v

cramers_v_table


,Low,Medium,High
Low,2.236068,1.414214,0.0
Medium,1.414214,2.236068,1.732051
High,0.0,1.732051,2.236068


## 3. Вычисление среднего значения индекса Кетле

Коэффициент Кетле - мера силы ассоциации между двумя категориальными переменными. Значения близкие к 0 указывают на слабую ассоциацию, а значения близкие к 1 на сильную.

In [ ]:
# Шаг 3: Вычисление среднего значения индекса Кетле для обеих таблиц сопряженности
average_cramers_v = (cramers_v_x1_x2 + cramers_v_x1_x3) / 2
average_cramers_v

0.43961554653995727

Среднее значение индекса Кетле: Приблизительно 0.44, указывает на умеренную связь между регионами и категориями подтвержденных случаев и смертей.

Сравнение коэффициентов Кетле

Если cramers_v_x1_x2 > cramers_v_x1_x3, это означает, что связь между регионом ВОЗ (x1) и категориями подтвержденных случаев COVID-19 (x2) сильнее, чем связь между регионом ВОЗ и категориями смертей (x3).

Если cramers_v_x1_x3 > cramers_v_x1_x2, то связь между регионом ВОЗ и категориями смертей сильнее.

Причины более сильной связи могут включать в себя различия в отчетности о случаях и смертях, политику здравоохранения и меры реагирования на COVID-19, а также другие социальные, экономические и географические факторы, влияющие на распространение вируса и его летальность в разных регионах.

## 4. Расчет коэффициента хи-квадрат

In [ ]:
# Шаг 4: Расчет коэффициента хи-квадрат для обеих таблиц
chi2_x1_x2, p_x1_x2, _, _ = chi2_contingency(contingency_table_x1_x2)
chi2_x1_x3, p_x1_x3, _, _ = chi2_contingency(contingency_table_x1_x3)

chi2_x1_x2, chi2_x1_x3, p_x1_x2, p_x1_x3


(31.795414172080793,
 40.76263342001429,
 0.0004331082099355789,
 1.2429827968990875e-05)

In [ ]:
print('Хи-квадрат =', chi2_x1_x2, 'p-значение =', p_x1_x2)
print('Хи-квадрат =', chi2_x1_x3, 'p-значение =', p_x1_x3)

Таблица сопряженности х1 и х2: Хи-квадрат = ~31.80, p-значение = 0.000433, указывая на значимую связь.

Таблица сопряженности х1 и х3: Хи-квадрат = ~40.76, p-значение = ~0.0000124, также указывая на значимую связь.

Общие выводы

Обе таблицы сопряженности показывают статистически значимую связь между регионами ВОЗ и категориями подтвержденных случаев и смертей.
Индекс Кетле указывает на наличие умеренной связи между этими переменными.
Результаты хи-квадрат подтверждают, что эти связи статистически значимы.
Связь между регионами ВОЗ и смертями (х1 и х3) кажется более выраженной, чем между регионами и подтвержденными случаями (х1 и х2), основываясь на значениях хи-квадрат и индекса Кетле.